# Uvoz podatkov

In [1]:
!head tabela-zivil.dat 

#vrednosti	za	100g	vsakega	zivila					
#zivilo	energija[kcal]	mascobe[g]	ogljikovihidrati[g]	proteini[g]	Ca[mg]	Fe[mg]	VitaminC[mg]	Kalij[mg]	Natrij[mg]	Cena[€/100g]
Ovseni_kosmici	369	6.1	64	15.5	54	4.7	0	354	286	0.15
Jabolko	52	0.17	13.81	0.26	6	0.12	4.6	107	1	0.15
Pomfri	93	12	21.55	1.96	5	0.35	12.8	391	241	0.99
Govedina	254	20	0	17.17	18	1.94	0	295	66	0.593
Svinjina	236	17.99	0	17.18	15	1.05	0.7	302	65	0.622
Piscanec	114	1.65	0	23.2	12	0.73	0	239	68	0.299
Mleko	60	3.25	4.52	3.22	113	0.03	0	113	40	0.085
Sir_edamec	357	27.8	1.43	24.99	731	0.44	0	188	965	1.266


In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
fajl = !ls *.dat
fajl

['tabela-zivil.dat']

In [5]:
zivila = pd.read_table(fajl[0], skiprows=1, sep="\t")
print("Imamo kak manjkajoč podatek?")
print(zivila.isna().any())
zivila.head()

Imamo kak manjkajoč podatek?
#zivilo                False
energija[kcal]         False
mascobe[g]             False
ogljikovihidrati[g]    False
proteini[g]            False
Ca[mg]                 False
Fe[mg]                 False
VitaminC[mg]           False
Kalij[mg]              False
Natrij[mg]             False
Cena[€/100g]           False
dtype: bool


,#zivilo,energija[kcal],mascobe[g],ogljikovihidrati[g],proteini[g],Ca[mg],Fe[mg],VitaminC[mg],Kalij[mg],Natrij[mg],Cena[€/100g]
0,Ovseni_kosmici,369,6.10,64.00,15.50,54,4.70,0.0,354,286.0,0.150
1,Jabolko,52,0.17,13.81,0.26,6,0.12,4.6,107,1.0,0.150
2,Pomfri,93,12.00,21.55,1.96,5,0.35,12.8,391,241.0,0.990
3,Govedina,254,20.00,0.00,17.17,18,1.94,0.0,295,66.0,0.593
4,Svinjina,236,17.99,0.00,17.18,15,1.05,0.7,302,65.0,0.622


# Prvi scenarij: 
"  Minimiziraj količino kalorij, če je priporočen minimalni dnevni
vnos  70 g  maščob,  310 g  ogljikovih  hidratov,  50 g  proteinov,  1000 mg  kalcija  ter  18 mg  železa. Upoštevaj tudi, da naj dnevni obroki količinsko ne presežejo dveh kilogramov hrane."

In [8]:
from scipy.optimize import linprog

In [9]:
c = zivila["energija[kcal]"]
b_ub = np.array([
    -70,
    -310,
    -50,
    -1000,
    -18,
    20
])
A = np.array([
    -zivila["mascobe[g]"],
    -zivila["ogljikovihidrati[g]"],
    -zivila["proteini[g]"],
    -zivila["Ca[mg]"],
    -zivila["Fe[mg]"],
    np.ones((len(zivila)))
])

X = linprog(c, A_ub = A, b_ub = b_ub)
X

     fun: 1288.991841997811
 message: 'Optimization terminated successfully.'
     nit: 14
   slack: array([  0.        ,   0.        ,  47.6545134 ,   0.        ,
        55.13152445,   0.        ])
  status: 0
 success: True
       x: array([  0.        ,   0.        ,   0.17491782,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.56219264,   0.        ,   4.95627636,
        14.30661317,   0.        ,   0.        ,   0.        ])

In [10]:
izpisi2(X)

Vrednost optimizirane funkcije:  1288.991842


Živilo,Količina[kg]
Sol,1.43066
Kakav,0.495628
Marmelada,0.0562193
Pomfri,0.0174918


Definiramo funkcijo, ki nam bo lepo izpisala rezultat

In [169]:
a = zivila.style.hide_index()
a.to_latex()

AttributeError: 'Styler' object has no attribute 'to_latex'

In [6]:
def izpisi2(x, zivila=zivila["#zivilo"]):
    print("Vrednost optimizirane funkcije: ", x.fun)
    df = pd.DataFrame(data={"Živilo": zivila, "Količina[kg]": 0.1*x.x}).sort_values("Količina[kg]",ascending=False)
    filter = df["Količina[kg]"] != 0
    return df[filter].style.bar(subset=["Količina[kg]"], color='#d65f5f').hide_index()


def izpisi(x, zivila=zivila["#zivilo"]):
    print("Vrednost optimizirane funkcije: ", x.fun)
    df = pd.DataFrame(data={"Živilo": zivila, "Količina[kg]": 0.1*x.x}).sort_values("Količina[kg]",ascending=False)
    filter = df["Količina[kg]"] != 0    
    print(df[filter].to_latex(decimal=",", index=False))
izpisi2(X)

NameError: name 'X' is not defined

In [12]:
zivila[zivila["#zivilo"] == "Sol"]

,#zivilo,energija[kcal],mascobe[g],ogljikovihidrati[g],proteini[g],Ca[mg],Fe[mg],VitaminC[mg],Kalij[mg],Natrij[mg],Cena[€/100g]
44,Sol,0,0.0,0.0,0.0,24,0.3,0.0,8,38758.0,0.109


## Drugi scenarij:
"Upoˇstevate lahko ˇse minimalne vnose za vitamin C (60 mg), kalij (3500 mg) in sprejemljiv interval
za natrij (500 mg – 2400 mg), ki so tudi na voljo v tabeli"

popravim A in b


In [22]:
c = zivila["energija[kcal]"]
b_ub = np.array([-70,-310,-50,-1000,-18,20, -60,-3500,2400,-500])
A = np.array([
    -zivila["mascobe[g]"],
    -zivila["ogljikovihidrati[g]"],
    -zivila["proteini[g]"],
    -zivila["Ca[mg]"],
    -zivila["Fe[mg]"],
    np.ones((len(zivila))),
    -zivila["VitaminC[mg]"],
    -zivila["Kalij[mg]"],
    zivila["Natrij[mg]"],
    -zivila["Natrij[mg]"]
    
])

X2 = linprog(c, A_ub = A, b_ub = b_ub)
X2

     fun: 1297.2737079300357
 message: 'Optimization terminated successfully.'
     nit: 37
   slack: array([    0.        ,     0.        ,    51.74546704,     0.        ,
          54.63526427,     0.        ,     0.        ,  4699.48779986,
           0.        ,  1900.        ])
  status: 0
 success: True
       x: array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.79670773,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.96621125,   0.        ,   0.        ,   0.        ,
         0.        ,   0.2973317 ,   0.        , 

In [172]:
izpisi(X2)

Vrednost optimizirane funkcije:  1297.27370793
\begin{tabular}{lr}
\toprule
    Živilo &  Količina[kg] \\
\midrule
  Radenska &      1,280673 \\
     Kakav &      0,508940 \\
 Pomaranca &      0,096621 \\
    Solata &      0,079671 \\
 Marmelada &      0,029733 \\
       Sol &      0,004362 \\
\bottomrule
\end{tabular}



## Tretji scenarij:

 Kako se rezultat razlikuje, ˇce zahtevamo minimalno 2000 kcal in namesto energije minimiziramo
vnos maˇsˇcob?

Najprej brez drugih vezi:

In [173]:

c = zivila["mascobe[g]"]
b_ub = np.array([-2000])
A = np.array([
    -zivila["energija[kcal]"]
    
])

X3 = linprog(c, A_ub = A, b_ub = b_ub)
izpisi(X3)

Vrednost optimizirane funkcije:  -0.0
\begin{tabular}{lr}
\toprule
Živilo &  Količina[kg] \\
\midrule
   Med &      0,657895 \\
\bottomrule
\end{tabular}



Haha, zakon... OK, pa če upoštevamo še use ostale vezi ka sm jih jemu prej?

In [174]:
c = zivila["mascobe[g]"]
b_ub = np.array([-310,
                 -50,
                 -1000,
                 -18,
                 20, 
                 -60,
                 -3500,
                 2400,
                 -500, 
                 -2000
                ])
A = np.array([
    -zivila["ogljikovihidrati[g]"],
    -zivila["proteini[g]"],
    -zivila["Ca[mg]"],
    -zivila["Fe[mg]"],
    np.ones((len(zivila))),
    -zivila["VitaminC[mg]"],
    -zivila["Kalij[mg]"],
    zivila["Natrij[mg]"],
    -zivila["Natrij[mg]"],
    -zivila["energija[kcal]"]    
])

X4 = linprog(c, A_ub = A, b_ub = b_ub)
izpisi(X4)

Vrednost optimizirane funkcije:  7.43766292266
\begin{tabular}{lr}
\toprule
    Živilo &  Količina[kg] \\
\midrule
    Solata &      1,156932 \\
 Marmelada &      0,490572 \\
     Fizol &      0,245238 \\
  Kruh\_bel &      0,102225 \\
     Puran &      0,005033 \\
\bottomrule
\end{tabular}



# 5 del; omejitev količine posameznih živil

Postopam enako, le da dodam še enotske vektorje v prostoru živil in v b_up dodam normalnejšo količino.

In [39]:
d = len(zivila)
kopija = zivila.copy()

for i, hr in enumerate(kopija["#zivilo"].values):
    kopija[hr] = np.zeros(d)
    kopija[hr][i] = 1

/home/peter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
kopija.tail()

,#zivilo,energija[kcal],mascobe[g],ogljikovihidrati[g],proteini[g],Ca[mg],Fe[mg],VitaminC[mg],Kalij[mg],Natrij[mg],...,Mortadela,Postrv,Olivno_olje,Marmelada,Med,Kakav,Sol,Radenska,Vino_cabernet,Vino_traminec
43,Kakav,229,13.7,54.3,19.60,128,13.86,0.0,1524,21.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
44,Sol,0,0.0,0.0,0.00,24,0.30,0.0,8,38758.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
45,Radenska,0,0.0,0.0,0.00,20,0.10,0.0,8,44.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
46,Vino_cabernet,84,0.0,2.6,0.07,0,0.00,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
47,Vino_traminec,82,0.0,2.6,0.07,0,0.00,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [43]:
c = kopija["energija[kcal]"]
b_ub = np.array([-70,
                 -310,
                 -50,
                 -1000,
                 -18,
                 20, 
                 -60,
                 -3500,
                 2400,
                 -500,
                 1, #Solata v enotah 100g
                 0.5, #Marmelada
                 2,
                 1,
                 2,
                 2,
                 1,
                 4,
                4,
                 0.1,
                 3,
                 2,
                 1
                ])
A = np.array([
    -kopija["mascobe[g]"],
    -kopija["ogljikovihidrati[g]"],
    -kopija["proteini[g]"],
    -kopija["Ca[mg]"],
    -kopija["Fe[mg]"],
    np.ones((len(kopija))),
    -kopija["VitaminC[mg]"],
    -kopija["Kalij[mg]"],
    kopija["Natrij[mg]"],
    -kopija["Natrij[mg]"],
    kopija["Solata"],
    kopija["Marmelada"],
    kopija["Pomaranca"],
    kopija["Zelje"],
    kopija["Brokoli"],
    kopija["Strocji_fizol"],
    kopija["Korenje"],
    kopija["Kruh_bel"],
    kopija["Radenska"],
    kopija["Kakav"],
    kopija["Pomfri"],
    kopija["Jagode"],
    kopija["Paprika"]
])

X5 = linprog(c, A_ub = A, b_ub = b_ub)
izpisi(X5)

Vrednost optimizirane funkcije:  1658.92854735
\begin{tabular}{llr}
\toprule
{} &         Živilo &  Količina[kg] \\
\midrule
2  &         Pomfri &      0,300000 \\
29 &        Brokoli &      0,200000 \\
36 &      Pomaranca &      0,200000 \\
21 &         Jagode &      0,200000 \\
34 &  Strocji\_fizol &      0,200000 \\
45 &       Radenska &      0,192389 \\
8  &       Kruh\_bel &      0,131007 \\
24 &          Torta &      0,112841 \\
32 &        Paprika &      0,100000 \\
33 &        Korenje &      0,100000 \\
17 &         Solata &      0,100000 \\
18 &          Zelje &      0,100000 \\
41 &      Marmelada &      0,027502 \\
7  &     Sir\_edamec &      0,026032 \\
43 &          Kakav &      0,010000 \\
44 &            Sol &      0,000229 \\
\bottomrule
\end{tabular}



In [175]:
izpisi(X5)

Vrednost optimizirane funkcije:  1658.92854735
\begin{tabular}{lr}
\toprule
        Živilo &  Količina[kg] \\
\midrule
        Pomfri &      0,300000 \\
       Brokoli &      0,200000 \\
     Pomaranca &      0,200000 \\
        Jagode &      0,200000 \\
 Strocji\_fizol &      0,200000 \\
      Radenska &      0,192389 \\
      Kruh\_bel &      0,131007 \\
         Torta &      0,112841 \\
       Paprika &      0,100000 \\
       Korenje &      0,100000 \\
        Solata &      0,100000 \\
         Zelje &      0,100000 \\
     Marmelada &      0,027502 \\
    Sir\_edamec &      0,026032 \\
         Kakav &      0,010000 \\
           Sol &      0,000229 \\
\bottomrule
\end{tabular}



# Part 7: Nizko-OH dieta
minimiziram OH intake ob omejitvi 2000kcal
The Dietary Guidelines for Americans by the United States Department of Agriculture (USDA) recommends three healthy patterns of diet, summarized in table below, for a 2000 kcal diet.[10]

In [180]:
c = zivila["ogljikovihidrati[g]"]
b_ub = np.array([-70, -50,-1000,-18,20, -60,-3500,2400,-500])
A = np.array([
    -zivila["mascobe[g]"],
    -zivila["proteini[g]"],
    -zivila["Ca[mg]"],
    -zivila["Fe[mg]"],
    np.ones((len(zivila))),
    -zivila["VitaminC[mg]"],
    -zivila["Kalij[mg]"],
    zivila["Natrij[mg]"],
    -zivila["Natrij[mg]"]
    
])
b_eq = [2000]
A_eq = [zivila["energija[kcal]"]]

X7 = linprog(c, A_ub = A, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq )
izpisi2(X7)
izpisi(X7)

Vrednost optimizirane funkcije:  2.49232742165
Vrednost optimizirane funkcije:  2.49232742165
\begin{tabular}{lr}
\toprule
     Živilo &  Količina[kg] \\
\midrule
     Postrv &      1,238202 \\
   Radenska &      0,685145 \\
 Sir\_edamec &      0,044690 \\
    Paprika &      0,029324 \\
        Sol &      0,002639 \\
\bottomrule
\end{tabular}



In [176]:
izpisi(X7)

Vrednost optimizirane funkcije:  2.49232742165
\begin{tabular}{lr}
\toprule
     Živilo &  Količina[kg] \\
\midrule
     Postrv &      1,238202 \\
   Radenska &      0,685145 \\
 Sir\_edamec &      0,044690 \\
    Paprika &      0,029324 \\
        Sol &      0,002639 \\
\bottomrule
\end{tabular}



In [51]:
sum(X7.x)

20.000000000000014

Vidimo, da bo potrebno omejiti živila (zdaj omejim energijo le navzgor na 2Mcal):

In [179]:
zivila = kopija
c = zivila["ogljikovihidrati[g]"]
b_ub = np.array([-70, -50,-1000,-18,20, -60,-3500,2400,-500, 6, 1,0.5,2,1,2,2,1,4,4,0.1,3,2,1,2000,6])
A = np.array([
    -zivila["mascobe[g]"],
    -zivila["proteini[g]"],
    -zivila["Ca[mg]"],
    -zivila["Fe[mg]"],
    np.ones((len(zivila))),
    -zivila["VitaminC[mg]"],
    -zivila["Kalij[mg]"],
    zivila["Natrij[mg]"],
    -zivila["Natrij[mg]"],
    zivila["Postrv"],
    zivila["Solata"],
    zivila["Marmelada"],
    zivila["Pomaranca"],
    zivila["Zelje"],
    zivila["Brokoli"],
    zivila["Strocji_fizol"],
    zivila["Korenje"],
    zivila["Kruh_bel"],
    zivila["Radenska"],
    zivila["Kakav"],
    zivila["Pomfri"],
    zivila["Jagode"],
    zivila["Paprika"],
    zivila["energija[kcal]"],
    zivila["Puran"]
    
])


X10 = linprog(c, A_ub = A, b_ub = b_ub)
izpisi(X10)

Vrednost optimizirane funkcije:  3.08703091236
\begin{tabular}{lr}
\toprule
        Živilo &  Količina[kg] \\
\midrule
        Postrv &      0,600000 \\
         Puran &      0,600000 \\
      Radenska &      0,400000 \\
    Sir\_edamec &      0,078592 \\
      Govedina &      0,046077 \\
       Paprika &      0,031063 \\
 Skusa\_soljena &      0,005247 \\
           Sol &      0,001319 \\
\bottomrule
\end{tabular}



In [178]:
izpisi(X10)

Vrednost optimizirane funkcije:  -4.63865500938


ValueError: array length 49 does not match index length 48

# Paleo dieta

"""
EAT

    Grass-fed meats
    Fish/seafood
    Fresh fruits
    Fresh vegetables
    Eggs
    Nuts
    Seeds
    Healthy oils (olive, walnut, flaxseed, macadamia, avocado, coconut)

DON’T EAT

    Cereal grains
    Legumes (including peanuts)
    Dairy
    Refined sugar
    Potatoes
    Processed foods
    Overly salty foods
    Refined vegetable oils
    Candy/junk/processed food

""" vir: https://ultimatepaleoguide.com/paleo-diet-food-list/

In [199]:
zivila.index[zivila["#zivilo"].isin(paleo)]

Int64Index([ 1,  3,  4,  5, 11, 12, 16, 17, 18, 20, 21, 22, 27, 29, 30, 32, 33,
            34, 35, 36, 37, 39, 40, 44, 45, 46, 47],
           dtype='int64')

In [79]:
paleo = [ 'Jabolko', 'Govedina', 'Svinjina','Piscanec',
       'Skusa_soljena', 'Losos',
       'Rdeca_pesa', 'Solata', 'Zelje',  'Grozdje', 'Jagode',
       'Jajce',     'Banana',
        'Brokoli', 'Tuna',  'Paprika', 'Korenje',
         'Kumara', 'Pomaranca', 'Puran', 
       'Postrv', 'Olivno_olje',    'Sol',
        'Vino_cabernet', 'Vino_traminec']
kopija2 = zivila[zivila["#zivilo"].isin(paleo)]
kopija2

,#zivilo,energija[kcal],mascobe[g],ogljikovihidrati[g],proteini[g],Ca[mg],Fe[mg],VitaminC[mg],Kalij[mg],Natrij[mg],...,Mortadela,Postrv,Olivno_olje,Marmelada,Med,Kakav,Sol,Radenska,Vino_cabernet,Vino_traminec
1,Jabolko,52,0.17,13.81,0.26,6,0.12,4.6,107,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Govedina,254,20.00,0.00,17.17,18,1.94,0.0,295,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Svinjina,236,17.99,0.00,17.18,15,1.05,0.7,302,65.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Piscanec,114,1.65,0.00,23.20,12,0.73,0.0,239,68.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,Skusa_soljena,305,25.10,0.00,18.50,66,1.40,0.0,520,4450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,Losos,142,5.34,0.00,19.84,12,0.80,0.0,490,44.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,Rdeca_pesa,37,0.00,8.00,2.00,16,0.79,4.8,325,78.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,Solata,17,0.20,3.35,1.25,52,0.83,9.0,194,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,Zelje,25,0.10,5.80,1.28,40,0.47,36.6,170,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,Grozdje,69,0.00,18.00,1.00,10,0.40,3.6,191,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [184]:
c = kopija2["energija[kcal]"]
b_ub = np.array([-70,
                 -310,
                 -50,
                -1000,
                -18,
                 -60,
                 3500,
                 -500,
                 2400
                                 
                
                ])
A = np.array([
    -kopija2["mascobe[g]"],
    -kopija2["ogljikovihidrati[g]"],
    -kopija2["proteini[g]"],
    -kopija2["Ca[mg]"],
    -kopija2["Fe[mg]"],
    -kopija2["VitaminC[mg]"],
    -kopija2["Kalij[mg]"],
    -kopija2["Natrij[mg]"],
    kopija2["Natrij[mg]"]
])

X8 = linprog(c, A_ub = A, b_ub = b_ub)
izpisi(X8, zivila = kopija2["#zivilo"])

Vrednost optimizirane funkcije:  1857.06306256
\begin{tabular}{lr}
\toprule
      Živilo &  Količina[kg] \\
\midrule
     Paprika &      3,476048 \\
      Solata &      1,140690 \\
     Jabolko &      0,377267 \\
 Olivno\_olje &      0,059778 \\
         Sol &      0,005176 \\
\bottomrule
\end{tabular}



cene dobim iz https://trgovina.mercator.si/market/, razen postrvi, ki je pa iz https://www.tuscc.si/iskanje?q=postrv

In [187]:
c = -zivila["Cena[€/100g]"]
b_ub = np.array([-70,-310,-50,-1000,-18,20, -60,-3500,2400,-500])
A = np.array([
    -zivila["mascobe[g]"],
    -zivila["ogljikovihidrati[g]"],
    -zivila["proteini[g]"],
    -zivila["Ca[mg]"],
    -zivila["Fe[mg]"],
    np.ones((len(zivila))),
    -zivila["VitaminC[mg]"],
    -zivila["Kalij[mg]"],
    zivila["Natrij[mg]"],
    -zivila["Natrij[mg]"]
    
])
A_eq = np.array([
    zivila["energija[kcal]"]
])
b_eq = np.array([2000])

X9 = linprog(c, A_ub = A, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq)
izpisi(X9)

Vrednost optimizirane funkcije:  -30.6940896464
\begin{tabular}{lr}
\toprule
     Živilo &  Količina[kg] \\
\midrule
     Solata &      1,057054 \\
      Losos &      0,508260 \\
      Kakav &      0,297228 \\
        Med &      0,137370 \\
 Sir\_edamec &      0,000088 \\
\bottomrule
\end{tabular}



In [133]:
zivila2= pd.read_table(fajl[0], skiprows=1, sep="\t")
#zivila2.drop(columns=[0], axis=1)
zivila2.columns

Index(['#zivilo', 'energija[kcal]', 'mascobe[g]', 'ogljikovihidrati[g]',
       'proteini[g]', 'Ca[mg]', 'Fe[mg]', 'VitaminC[mg]', 'Kalij[mg]',
       'Natrij[mg]', 'Cena[€/100g]'],
      dtype='object')

In [153]:
zivila2.loc[48] = ["Pica", 246,9.5,27,13.5,198.4,0,0,0,512,0.5]


In [148]:
np.array(["Pica", 246,9.5,27,13.5,198.4,0,0,0,512,0.5]).shape

(11,)

In [157]:
d = len(zivila2)
kopija3 = zivila2.copy()

for i, hr in enumerate(kopija3["#zivilo"].values):
    kopija3[hr] = np.zeros(d)
    kopija3[hr][i] = 1
kopija3.tail()

/home/peter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,#zivilo,energija[kcal],mascobe[g],ogljikovihidrati[g],proteini[g],Ca[mg],Fe[mg],VitaminC[mg],Kalij[mg],Natrij[mg],...,Postrv,Olivno_olje,Marmelada,Med,Kakav,Sol,Radenska,Vino_cabernet,Vino_traminec,Pica
44,Sol,0,0.0,0.0,0.00,24.0,0.3,0.0,8,38758.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
45,Radenska,0,0.0,0.0,0.00,20.0,0.1,0.0,8,44.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
46,Vino_cabernet,84,0.0,2.6,0.07,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
47,Vino_traminec,82,0.0,2.6,0.07,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48,Pica,246,9.5,27.0,13.50,198.4,0.0,0.0,0,512.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [186]:
c = -kopija3["Pica"]
b_ub = np.array([-70,
                 -310,
                 -50,
                -1000,
                -18,
                 -60,
                 3500,
                 -500,
                 2400,
                 2
                
                ])
A = np.array([
    -kopija3["mascobe[g]"],
    -kopija3["ogljikovihidrati[g]"],
    -kopija3["proteini[g]"],
    -kopija3["Ca[mg]"],
    -kopija3["Fe[mg]"],
    -kopija3["VitaminC[mg]"],
    -kopija3["Kalij[mg]"],
    -kopija3["Natrij[mg]"],
    kopija3["Natrij[mg]"],
    kopija3["Pomaranca"]
])

A_eq = np.array([
    kopija3["energija[kcal]"]
])
b_eq = np.array([2000])



X10 = linprog(c, A_ub = A, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq)
izpisi(X10, zivila = kopija3["#zivilo"])

Vrednost optimizirane funkcije:  -4.63865500938
\begin{tabular}{lr}
\toprule
      Živilo &  Količina[kg] \\
\midrule
        Pica &      0,463866 \\
      Banana &      0,253553 \\
   Pomaranca &      0,200000 \\
         Riz &      0,160029 \\
       Kakav &      0,106063 \\
 Olivno\_olje &      0,009989 \\
\bottomrule
\end{tabular}



In [183]:
for i in [X, X2, X3, X4, X5, X7, X8, X9, X10]:
    try:
        izpisi(i)
    except Exception as e:
        print(i, e, sep=":  ")

Vrednost optimizirane funkcije:  1288.991842
\begin{tabular}{lr}
\toprule
    Živilo &  Količina[kg] \\
\midrule
       Sol &      1,430661 \\
     Kakav &      0,495628 \\
 Marmelada &      0,056219 \\
    Pomfri &      0,017492 \\
\bottomrule
\end{tabular}

Vrednost optimizirane funkcije:  1297.27370793
\begin{tabular}{lr}
\toprule
    Živilo &  Količina[kg] \\
\midrule
  Radenska &      1,280673 \\
     Kakav &      0,508940 \\
 Pomaranca &      0,096621 \\
    Solata &      0,079671 \\
 Marmelada &      0,029733 \\
       Sol &      0,004362 \\
\bottomrule
\end{tabular}

Vrednost optimizirane funkcije:  -0.0
\begin{tabular}{lr}
\toprule
Živilo &  Količina[kg] \\
\midrule
   Med &      0,657895 \\
\bottomrule
\end{tabular}

Vrednost optimizirane funkcije:  7.43766292266
\begin{tabular}{lr}
\toprule
    Živilo &  Količina[kg] \\
\midrule
    Solata &      1,156932 \\
 Marmelada &      0,490572 \\
     Fizol &      0,245238 \\
  Kruh\_bel &      0,102225 \\
     Puran &      0,005033 